# Demonstration of how to use CellSegmentationTracker


In this code, it will be demonstrated how to use CellSegmentationTracker and its methods. If you like, you can use the sample data file epi500.tif, which you can download from https://github.com/simonguld/CellSegmentationTracker/blob/main/resources/epi500_sample_images.tif. 

In this folder, you can also find the xml file generated by running the segmentation and tracking as well as the spots, tracks and edges csv files generated by CellSegmentationTracker.

In [4]:
import os

from cellsegmentationtracker import CellSegmentationTracker

In the following, we will walk through the class parameters one by one and finally initialize an instance of CellSegmentationTracker:

In [12]:
## Set paths to executables:
#  If you want to do segmentation and tracking, you need to provide paths to ImageJ and Cellpose. 
# If you already have an xml file and just want to use CellSegmentationTracker to generate csv files, 
# calculate velocities or analyse the results, you don't have to provide these
cellpose_python_filepath = 'C:\\Users\\Simon Andersen\\miniconda3\\envs\\cellpose\\python.exe'
imj_path = "C:\\Users\\Simon Andersen\\Fiji.app\\ImageJ-win64.exe"

# Set path to .tif image file (or folder with images). If you already have an xml file, you don't have to provide it
image_path = "C:\\Users\\Simon Andersen\\Projects\\Projects\\CellSegmentationTracker\\resources\\epi500_sample_images.tif"

# If you have already done segmentation and tracking, you can simply provide the path to the xml file.
xml_path = None 

# Set path to output folder. If None, results will be outputted in the same folder as the image.
output_folder_path = None

# Set whether to use the pretrained model or not. If not, you need to provide the path to a custom model
use_model = 'EPI500'
custom_model_path = None

# Set whether to open ImageJ and show the segmentation and tracking results
show_segmentation = False
# Set cellpose and trackmate settings. If you don't provide any, the default settings will be used
cellpose_dict = {
            'TARGET_CHANNEL' : 0,
            'OPTIONAL_CHANNEL_2': 0,
            'FLOW_THRESHOLD': 0.4,
            'CELLPROB_THRESHOLD': 0.5,
            'CELL_DIAMETER': 0.0, # If 0.0, the diameter will be estimated by Cellpose
            'USE_GPU': False,
            'SIMPLIFY_CONTOURS': True
            }
# Beware that if you set ALLOW_TRACK_SPLITTING and/or ALLOW_TRACK_MERGING to True, the calculated velocities might be incorrect, as several 
# cells will be merged into one track and can be present at the same time
trackmate_dict = {'LINKING_MAX_DISTANCE': 15.0,
                                         'GAP_CLOSING_MAX_DISTANCE': 15.0,
                                         'MAX_FRAME_GAP': 2,
                                         'ALLOW_TRACK_SPLITTING': False, 
                                         'ALLOW_TRACK_MERGING': False,
             }


# Now having set all parameters, we are ready to initialise the CellSegmentationTracker object
cst = CellSegmentationTracker(imagej_filepath = imj_path, cellpose_python_filepath = cellpose_python_filepath, \
                              image_folder_path = image_path, xml_path = xml_path, output_folder_path = output_folder_path,
                  use_model = use_model, custom_model_path = custom_model_path, show_segmentation = show_segmentation, cellpose_dict = dict(), trackmate_dict = dict(),)



In [13]:
# To run the segmentation and tracking, simply call the run_segmentation_tracking() method. 
# If you are using the epi500 sample images (without GPU), it will take 2-3 minutes to run
cst.run_segmentation_tracking()

Computing spot features over 1 frame simultaneously and allocating 8 threads per frame.
Calculating 0 spots features...

Computation done in 13 ms.
Computing track features:
  - Branching analyzer in 0 ms.
  - Track duration in 0 ms.
  - Track index in 0 ms.
  - Track location in 0 ms.
  - Track speed in 0 ms.
  - Track quality in 0 ms.
  - Track motility analysis in 0 ms.
Computation done in 1 ms.
>>>> !NEW LOGGING SETUP! To see cellpose progress, set --verbose
No --verbose => no progress or info printed
Starting detection process using 8 threads.
Detection processes 8 frames simultaneously and allocates 1 thread per frame.
Detection...
Found 4550 spots.

Results saved to: C:\Users\Simon Andersen\Projects\Projects\CellSegmentationTracker\resources\epi500_sample_images.xml

Tracking and segmentation completed.



In [14]:
# After the segmentation and tracking is done, you can generate csv files with the results by calling the generate_csv_files() method

# Spot features will be extracted automatically. Decide whether to calculate cell velocities, as well as track and edge features. By default, everything is extracted and saved
calculate_velocities = True
get_tracks = True
get_edges = True

# If you want to save the csv files, set save_csv_files to True and provide a name for the csv files. If None, the name will be the same as the image name.
# The file(s) will be saved in the output folder if provided, otherwise in the same folder as the image if provided, otherwise in the current working directory
cst.generate_csv_files(calculate_velocities = calculate_velocities, get_tracks = get_tracks, get_edges = get_edges, save_csv_files = True, name = None)

# Now, you can access the spots, tracks and edges dataframes as follows:
# The final two columns of the spots object are the velocities (if calculated) in the x and y direction, respectively
df_spots, df_tracks, df_edges = cst.spots_df, cst.tracks_df, cst.edges_df

df_spots.info()

Starting to generate csv files from xml file now. This may take a while...                   
Processing an XML file with 120.000 spots, 90.000 edges and 20.000 tracks takes about 6-7 minutes to process on a regular labtop.
Saved spots csv file to:  C:\Users\Simon Andersen\Projects\Projects\CellSegmentationTracker\resources\epi500_sample_images_spots.csv
Saved tracks csv file to:  C:\Users\Simon Andersen\Projects\Projects\CellSegmentationTracker\resources\epi500_sample_images_tracks.csv
Saved edges csv file to:  C:\Users\Simon Andersen\Projects\Projects\CellSegmentationTracker\resources\epi500_sample_images_edges.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4550 entries, 0 to 4549
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Frame                   4550 non-null   float64
 1   T                       4550 non-null   float64
 2   X                       4550 non-null   float64
 3 

In [15]:
# To get basic settings and information about the segmentation and tracking, you can call the print_settings() method
cst.print_settings()


Image information:  ['filename: epi500_sample_images.tif', 'folder: C:\\Users\\Simon Andersen\\Projects\\Projects\\CellSegmentationTracker\\resources\\', 'width: 698', 'height: 648', 'nslices: 1', 'nframes: 11', 'pixelwidth: 1.0', 'pixelheight: 1.0', 'voxeldepth: 1.0', 'timeinterval: 1.0', 'xstart: 0', 'xend: 697', 'ystart: 0', 'yend: 647', 'zstart: 0', 'zend: 0', 'tstart: 0', 'tend: 10']
Cellpose settings:  {'TARGET_CHANNEL': 0, 'OPTIONAL_CHANNEL_2': 0, 'CELLPOSE_PYTHON_FILEPATH': 'C:\\Users\\Simon Andersen\\miniconda3\\envs\\cellpose\\python.exe', 'CELLPOSE_MODEL': 'EPI500', 'CELL_DIAMETER': 0.0, 'USE_GPU': False, 'SIMPLIFY_CONTOURS': True, 'CELLPOSE_MODEL_FILEPATH': 'C:\\Users\\Simon Andersen\\Projects\\Projects\\CellSegmentationTracker\\models\\epi500', 'FLOW_THRESHOLD': 0.4, 'CELLPROB_THRESHOLD': 0.5}
Trackmate settings:  {'LINKING_MAX_DISTANCE': 15.0, 'GAP_CLOSING_MAX_DISTANCE': 15.0, 'MAX_FRAME_GAP': 2, 'ALLOW_TRACK_SPLITTING': False, 'ALLOW_TRACK_MERGING': False} 

